In [1]:
import torch
import torch.nn as nn
from functorch import make_functional, vmap, vjp, jvp, jacrev
device = 'cpu'

class MLP(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.fc1(x)
        x = x.relu()
        x = self.fc2(x)
        x = x.relu()
        x = self.fc3(x)
        x = x.flatten(1)
        return x
    
model = MLP().to(device)

func, params = make_functional(model)

/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_843/3976059310.py:24: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  func, params = make_functional(model)


In [2]:
def fnet_single(params, x):
    return func(params, x.unsqueeze(0)).squeeze(0)

def empirical_ntk(fnet_single, params, x1, x2):
    # Compute J(x1)
    jac1 = vmap(jacrev(fnet_single), (None, 0))(params, x1)
    jac1 = [j.flatten(2) for j in jac1]
    
    # Compute J(x2)
    jac2 = vmap(jacrev(fnet_single), (None, 0))(params, x2)
    jac2 = [j.flatten(2) for j in jac2]
    
    # Compute J(x1) @ J(x2).T
    result = torch.stack([torch.einsum('Naf,Mbf->NMab', j1, j2) for j1, j2 in zip(jac1, jac2)])
    result = result.sum(0)
    return result

In [3]:
x_train = torch.randn(1, 128)
x_test = torch.randn(5, 128)

jac1 = jacrev(fnet_single, argnums=0)(params, x_train)

/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_843/4005020155.py:4: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jacrev` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jacrev` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  jac1 = jacrev(fnet_single, argnums=0)(params, x_train)


In [4]:
for params in model.named_parameters():
    print(params[0], params[1].size())

fc1.weight torch.Size([256, 128])
fc1.bias torch.Size([256])
fc2.weight torch.Size([512, 256])
fc2.bias torch.Size([512])
fc3.weight torch.Size([10, 512])
fc3.bias torch.Size([10])


In [7]:
print(len(jac1))

for j in list(jac1):
    print(j.size())


6
torch.Size([10, 256, 128])
torch.Size([10, 256])
torch.Size([10, 512, 256])
torch.Size([10, 512])
torch.Size([10, 10, 512])
torch.Size([10, 10])


In [2]:
import torch
import torch.nn as nn
from torch.autograd.functional import jacobian

x_train = torch.randn(1, 128)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

model = MLP()


output = model(x_train)
fc2_weights = model.fc2.weight

def forward_with_respect_to_fc2(fc2_weights):
    original_weights = model.fc2.weight.clone()
    with torch.no_grad():
        model.fc2.weight.copy_(fc2_weights)
    return model(x_train)

jac_fc2 = jacobian(forward_with_respect_to_fc2, fc2_weights)
print(jac_fc2.shape)  


RuntimeError: During a grad (vjp, jvp, grad, etc) transform, the function provided attempted to call in-place operation (aten::copy_) that would mutate a captured Tensor. This is not supported; please rewrite the function being transformed to explicitly accept the mutated Tensor(s) as inputs.

In [2]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from torch.autograd.functional import jacobian

model = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-14m')
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-14m')
tokenizer.pad_token = tokenizer.eos_token

input_text = "The dog is happy"

inputs = tokenizer(input_text, return_tensors='pt', max_length = 10, padding = "max_length")

output = model(**inputs)
param = model.gpt_neox.layers[4].mlp.dense_4h_to_h.weight

def forward_with_respect_to_fc2(fc2_weights):
    original_weights = model.gpt_neox.layers[4].mlp.dense_4h_to_h.weight.clone()
    with torch.no_grad():
        model.gpt_neox.layers[4].mlp.dense_4h_to_h.weight.copy_(fc2_weights)
    return model(**inputs).logits

jac_fc2 = jacobian(forward_with_respect_to_fc2, param)

/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_8101/386323017.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.jacrev` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.jacrev` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  jac_fc2 = vmap(jacrev(forward_with_respect_to_fc2))(param)
/var/folders/_x/mkf1szd12yv28px5ztl_sjjr0000gn/T/ipykernel_8101/386323017.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  jac_fc2 = vmap

NotImplementedError: Cannot access storage of TensorWrapper